In [15]:
import pandas as pd
import json
import re

In [16]:
data = pd.read_json('test.json')
data = pd.DataFrame(data)

testData = data.loc[0, 'text']
testData = str(testData)

In [14]:
p=re.compile(r'\b[Aa]ls\b')
#p.findall(testData)
#len(p.findall(testData))

In [13]:
# finds locations (start, end) of all matches
p=re.compile(r'als')
iterator = p.finditer(testData)

positions = []

for match in iterator:
    newPosition = match.start()
    positions.append(newPosition)

#positions

In [4]:
output = data
output['matches_schop'] = ''
output['pos_schop'] = ''
output['frac_schop'] = ''

for i, row in output.iterrows():
    chunk = row['text']
    chunk = str(chunk)

    pAll=re.compile(r'.*')
    iterAll = pAll.finditer(chunk)
    for match in iterAll:
        span = match.start()
    output.at[i, 'length'] = span

    pSchop = re.compile(r'\b[Ss]chopenhauer\w+') # will match all words beginning with 'Schopenhauer' or 'schopenhauer'
    output.at[i, 'matches_schop'] = pSchop.findall(chunk)
    output.at[i, 'num_schop'] = len(pSchop.findall(chunk))
    
    positions = []  
    fractions = []
    iterSchop = pSchop.finditer(chunk)
    for match in iterSchop:
        newPosition = match.start()
        newFraction = newPosition/span
        positions.append(newPosition)
        fractions.append(newFraction)

    if (positions == []):
        output.at[i, 'pos_schop'] = ''
    else:
        output.at[i, 'pos_schop'] = positions

    if(fractions == []):
        output.at[i, 'frac_schop'] = ''
    else: 
        output.at[i, 'frac_schop'] = fractions

output

,year,group,text,matches_schop,pos_schop,frac_schop,length,num_schop
0,1869,1,[Wer heutzutage von Aeschylus Sophocles Euripi...,[],,,50467.0,0.0
1,1869,2,"[Das Christenthum, ursprünglich Sache des Tale...",[],,,17195.0,0.0
2,1869,3,[Wir sind leider gewöhnt die Künste in der Ver...,[Schopenhauersche],[16071],[0.42369038517307744],37931.0,1.0
3,1870,4,"[Erlangen Samstag den 20. August., Hier seit a...",[],,,6619.0,0.0
4,1870,5,[Gedanken zu\n„die Tragoedie und die\nFreigeis...,[schopenhauerischen],[21781],[0.4785034820624355],45519.0,1.0
...,...,...,...,...,...,...,...,...
171,1888,21,"[Teichm<üller>, Scept<iques> gr<ecs>, Spir, A<...",[],,,4258.0,0.0
172,1888,22,[Randbemerkung zu einer niaserie anglaise. — „...,[],,,18594.0,0.0
173,1888,23,[Auch ein Gebot der Menschenliebe. — Es giebt ...,[],,,23305.0,0.0
174,1888,24,[Ecce homo\nOder:\nwarum ich Einiges mehr weis...,"[Schopenhauern, Schopenhauers]","[2394, 25747]","[0.06680992381324477, 0.7185276142103647]",35833.0,2.0


In [33]:
#reformatting so that svelte app can more easily work with data

## MUST MANUALLY DELETE data_jsonified.json AND PUT IN AN EMPTY []
array = data
array['matches_schop'] = ''
array['pos_schop'] = ''
array['frac_schop'] = ''
rowNum = 0

indexer = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]*9

def rowFinder(i):
    if i <= 19:
        rowNum = 0
    elif i > 19 and i <= 38:
        rowNum = 1
    elif i > 38 and i <= 57:
        rowNum = 2
    elif i > 57 and i <= 76:
        rowNum = 3
    elif i > 76 and i <= 95:
        rowNum = 4
    elif i > 95 and i <= 114:
        rowNum = 5
    elif i > 114 and i <= 133:
        rowNum = 6
    elif i > 133 and i <= 152:
        rowNum = 7
    elif i > 152 and i <= 171:
        rowNum = 8
    
    return rowNum


for i, row in array.iterrows():
    chunk = row['text']
    chunk = str(chunk)

    pAll=re.compile(r'.*')
    iterAll = pAll.finditer(chunk)
    for match in iterAll:
        span = match.start()
    text_length = span

    ## PEOPLE
    # Schopenhauer
    pSchop = re.compile(r'\b[Ss]chopenhauer\w+') 
    matches_schop = pSchop.findall(chunk)
    num_matches_schop = len(pSchop.findall(chunk))
    
    positions_schop = []  
    fractions_schop = []
    iterSchop = pSchop.finditer(chunk)
    for match in iterSchop:
        newPosition = match.start()
        newFraction = newPosition/span
        positions_schop.append(newPosition)
        fractions_schop.append(newFraction)

    # Wagner
    pWagn = re.compile(r'\b[Ww]agner\w+') 
    matches_wagn = pWagn.findall(chunk)
    num_matches_wagn = len(pWagn.findall(chunk))
    
    positions_wagn = []  
    fractions_wagn = []
    iterWagn = pWagn.finditer(chunk)
    for match in iterWagn:
        newPosition = match.start()
        newFraction = newPosition/span
        positions_wagn.append(newPosition)
        fractions_wagn.append(newFraction)

    # Kant
    pKant = re.compile(r'\b[Kk]ant\w+') 
    matches_kant = pKant.findall(chunk)
    num_matches_kant = len(pKant.findall(chunk))
    
    positions_kant = []  
    fractions_kant = []
    iterKant = pKant.finditer(chunk)
    for match in iterKant:
        newPosition = match.start()
        newFraction = newPosition/span
        positions_kant.append(newPosition)
        fractions_kant.append(newFraction)

    ## GENERAL
    # Deutschland
    pDeutsch = re.compile(r'\b[Dd]eutsch\w+') 
    matches_deutsch = pDeutsch.findall(chunk)
    num_matches_deutsch = len(pDeutsch.findall(chunk))
    
    positions_deutsch = []  
    fractions_deutsch = []
    iterDeutsch = pDeutsch.finditer(chunk)
    for match in iterDeutsch:
        newPosition = match.start()
        newFraction = newPosition/span
        positions_deutsch.append(newPosition)
        fractions_deutsch.append(newFraction)
    
    # musik
    pMusik = re.compile(r'\b[Mm]usik\w+') 
    matches_musik = pMusik.findall(chunk)
    num_matches_musik = len(pMusik.findall(chunk))
    
    positions_musik = []  
    fractions_musik = []
    iterMusik = pMusik.finditer(chunk)
    for match in iterMusik:
        newPosition = match.start()
        newFraction = newPosition/span
        positions_musik.append(newPosition)
        fractions_musik.append(newFraction)

    ## PHILOSOPHY
    # wille
    pWille = re.compile(r'\b [Ww]ille\w+') 
    matches_wille = pWille.findall(chunk)
    num_matches_wille = len(pWille.findall(chunk))
    
    positions_wille = []  
    fractions_wille = []
    iterWille = pWille.finditer(chunk)
    for match in iterWille:
        newPosition = match.start()
        newFraction = newPosition/span
        positions_wille.append(newPosition)
        fractions_wille.append(newFraction)
    
    # din an sich
    pDing = re.compile(r'\bDing an sich\w+') 
    matches_ding = pDing.findall(chunk)
    num_matches_ding = len(pDing.findall(chunk))
    
    positions_ding = []  
    fractions_ding = []
    iterDing = pDing.finditer(chunk)
    for match in iterDing:
        newPosition = match.start()
        newFraction = newPosition/span
        positions_ding.append(newPosition)
        fractions_ding.append(newFraction)
    
    # wahrheit
    pWahr = re.compile(r'\b [Ww]ahrheit\w+') 
    matches_wahr = pWahr.findall(chunk)
    num_matches_wahr = len(pWahr.findall(chunk))
    
    positions_wahr = []  
    fractions_wahr = []
    iterWahr = pWahr.finditer(chunk)
    for match in iterWahr:
        newPosition = match.start()
        newFraction = newPosition/span
        positions_wahr.append(newPosition)
        fractions_wahr.append(newFraction)

    # tragische
    pTragische = re.compile(r'\b [Tt]ragi\w+') 
    matches_tragische = pTragische.findall(chunk)
    num_matches_tragische = len(pTragische.findall(chunk))
    
    positions_tragische = []  
    fractions_tragische = []
    iterTragische = pTragische.finditer(chunk)
    for match in iterTragische:
        newPosition = match.start()
        newFraction = newPosition/span
        positions_tragische.append(newPosition)
        fractions_tragische.append(newFraction)
    
    # leiden
    pLeiden = re.compile(r'\b [Ll]eiden\w+') 
    matches_leiden = pLeiden.findall(chunk)
    num_matches_leiden = len(pLeiden.findall(chunk))
    
    positions_leiden = []  
    fractions_leiden = []
    iterLeiden = pLeiden.finditer(chunk)
    for match in iterLeiden:
        newPosition = match.start()
        newFraction = newPosition/span
        positions_leiden.append(newPosition)
        fractions_leiden.append(newFraction)
    
    ## RELIGION
    # Hindusimus
    pHindu = re.compile(r'\b [Hh]indus\w+') 
    pVedanta = re.compile(r'\b [Vv]eda\w+')
    matches_hindu = pHindu.findall(chunk) + pVedanta.findall(chunk)
    num_matches_hindu = len(pHindu.findall(chunk)) + len(pVedanta.findall(chunk))
    
    positions_hindu = []  
    fractions_hindu = []
    iterHindu = pHindu.finditer(chunk)
    for match in iterHindu:
        newPosition = match.start()
        newFraction = newPosition/span
        positions_hindu.append(newPosition)
        fractions_hindu.append(newFraction)
    
    # Buddhismus
    pBudd = re.compile(r'\b [Bb]uddhis\w+') 
    matches_budd = pBudd.findall(chunk)
    num_matches_budd = len(pBudd.findall(chunk))
    
    positions_budd = []  
    fractions_budd = []
    iterBudd = pBudd.finditer(chunk)
    for match in iterBudd:
        newPosition = match.start()
        newFraction = newPosition/span
        positions_budd.append(newPosition)
        fractions_budd.append(newFraction)

    # Schleier
    pSchleier = re.compile(r'\bSchleier\w+') 
    matches_schleier = pSchleier.findall(chunk)
    num_matches_schleier = len(pSchleier.findall(chunk))
    
    positions_schleier = []  
    fractions_schleier = []
    iterSchleier = pSchleier.finditer(chunk)
    for match in iterSchleier:
        newPosition = match.start()
        newFraction = newPosition/span
        positions_schleier.append(newPosition)
        fractions_schleier.append(newFraction)
    
    j = {
        'index': indexer[i],
        'row': rowFinder(i),
        'year': int(row['year']),
        'group': int(row['group']),
        'text': row['text'],
        'total_length': text_length,
        'schop_matches': {
            'num_matches': num_matches_schop,
            'matched_text': matches_schop,
            'matched_positions': positions_schop
        },
        'wagn_matches': {
            'num_matches': num_matches_wagn,
            'matched_text': matches_wagn,
            'matched_positions': positions_wagn
        },
        'kant_matches': {
            'num_matches': num_matches_wagn,
            'matched_text': matches_wagn,
            'matched_positions': positions_wagn
        },
        'deutch_matches': {
            'num_matches': num_matches_deutsch,
            'matched_text': matches_deutsch,
            'matched_positions': positions_deutsch
        },
        'musik_matches': {
            'num_matches': num_matches_musik,
            'matched_text': matches_musik,
            'matched_positions': positions_musik
        },
        'wille_matches': {
            'num_matches': num_matches_wille,
            'matched_text': matches_wille,
            'matched_positions': positions_wille
        },
        'ding_matches': {
            'num_matches': num_matches_ding,
            'matched_text': matches_ding,
            'matched_positions': positions_ding
        },
        'wahr_matches': {
            'num_matches': num_matches_wahr,
            'matched_text': matches_wahr,
            'matched_positions': positions_wahr
        },
        'tragische_matches': {
            'num_matches': num_matches_tragische,
            'matched_text': matches_tragische,
            'matched_positions': positions_tragische
        },
        'leiden_matches': {
            'num_matches': num_matches_leiden,
            'matched_text': matches_leiden,
            'matched_positions': positions_leiden
        },
        'hindu_matches': {
            'num_matches': num_matches_hindu,
            'matched_text': matches_hindu,
            'matched_positions': positions_hindu
        },
        'budd_matches': {
            'num_matches': num_matches_budd,
            'matched_text': matches_budd,
            'matched_positions': positions_budd
        },
        'schleier_matches': {
            'num_matches': num_matches_schleier,
            'matched_text': matches_schleier,
            'matched_positions': positions_schleier
        }
    }

    with open('data_jsonified.json', 'r') as file:
        a = json.load(file)
        a.append(j)
    with open('data_jsonified.json', 'w') as file:
        json.dump(a, file, indent=4)

    j = {}

NameError: name 'iterMusik' is not defined

In [ ]:
    # if num_matches = 0:

    # else:
    #     for i in num_matches:
    #         j = {
    #             'index': i,
    #             'year': int(row['year']),
    #             'group': int(row['group']),
    #             'text': row['text'],
    #             'length': text_length,
    #             'schop_matches': {
    #                 'num_matches': num_matches,
    #                 'matched_text': matches_schop,
    #                 'matched_positions': positions,
    #                 'matched_fractions': fractions,
    #                 'length': text_length
    #             }
    # }